In [1]:

import os

# Configuration
CONFIG = {
    'lambda_role_arn': 'arn:aws:iam::387311887840:role/financial-analysis-lambda-role',
    'step_functions_role_arn': 'arn:aws:iam::387311887840:role/financial-analysis-step-functions-role',
    'region': 'us-east-1'  # change to your region
}

# Run setup
print("Starting setup...")

# First, test AWS credentials
import boto3
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
print(f"Running setup for AWS Account: {account_id}")

Starting setup...
Running setup for AWS Account: 387311887840


In [5]:
# Import all setup classes
from infrastructure.aws_setup import AWSResourceSetup
from infrastructure.lambda_setup import LambdaSetup
from infrastructure.api_gateway_setup import APIGatewaySetup
from infrastructure.step_functions_setup import StepFunctionsSetup
from src.utils.security import setup_security
from src.data.dataset_preparation import DatasetPreparation
from config import AWS_CONFIG, API_KEYS, MODEL_CONFIG

In [ ]:
# Initialize setup
aws_setup = AWSResourceSetup()
lambda_setup = LambdaSetup()
api_setup = APIGatewaySetup()
step_functions_setup = StepFunctionsSetup()

In [ ]:
# Prepare dataset
data_prep = DatasetPreparation(f'financial-reports-raw-{account_id}')
training_data = data_prep.prepare_training_data()
print(f"Prepared training dataset with {len(training_data)} records")

In [ ]:
# Setup core infrastructure
print("Setting up security...")
security_config = setup_security()

print("\nSetting up S3 buckets...")
aws_setup.setup_s3()

print("\nSetting up DynamoDB tables...")
aws_setup.setup_dynamodb()

print("\nSetting up SQS queues...")
aws_setup.setup_sqs()

print("\nSetting up Lambda functions...")
lambda_setup.create_deployment_package()
lambda_setup.deploy_lambda(lambda_role_arn, queue_url)

print("\nSetting up Step Functions workflow...")
step_functions_setup.setup_workflow(step_functions_role_arn)

print("\nSetting up API Gateway...")
api_id = api_setup.setup_api()

print(f"Setting up infrastructure in region: {AWS_CONFIG['region']}")
print(f"Using bucket: {AWS_CONFIG['bucket_name']}")

print("\nSetup complete!")